In [5]:
# Installs all the necessary packages for the project
# %pip install pandas pandas-ta numpy matplotlib statsmodels pandas_datareader datetime yfinance scikit-learn PyPortfolioOpt
# %pip install --upgrade certifi

In [6]:
# Imports all the necessary packages for the project and fixes ssl error
import ssl
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
# Get SP500 data    
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = dt.datetime.now().strftime('%Y-%m-%d')
start_date = (pd.to_datetime(end_date) - pd.DateOffset(years=10)).strftime('%Y-%m-%d')

df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()

df.index.names = ['Date', 'Symbol']

df

[***************       31%%                      ]  154 of 503 completed

In [ ]:
# Calculate features and technical indicators
# Garman-Klass Volatility, RSI, Bollinger Bands, ATR, MACD, Dollar Volume,
# All but RSI are normalized by subtracting the mean and dividing by the standard deviation

df['Garman-Klass'] = ((np.log(df['High'])-np.log(df['Low']))**2)/2-(2*np.log(2)-1)*((np.log(df['Adj Close'])-np.log(df['Open']))**2)  

df['RSI'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.rsi(close=x, length=20))

df['BB-Low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['BB-Mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['BB-High'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(data):
    atr = ta.atr(high=data['High'],
                        low=data['Low'],
                        close=data['Close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())
df['ATR'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['MACD'] = df.groupby(level=1, group_keys=False)['Adj Close'].apply(compute_macd)

df['Dollar Volume'] = (df['Adj Close']*df['Volume'])/1e6

df


In [ ]:
# Aggregate to monthly data and filter top 150 most liquid stocks for each month

tech_cols = [c for c in df.columns.unique(0) if c not in ['Dollar Volume', 'Volume', 'Open',
                                                          'High', 'Low', 'Close']]

tech_data = (pd.concat([df.unstack('Symbol')['Dollar Volume'].resample('M').mean().stack('Symbol').to_frame('Dollar Volume'),
                   df.unstack()[tech_cols].resample('M').last().stack('Symbol')],
                  axis=1)).dropna()

# Calculate 5-year rolling average of Dollar Volume
tech_data['Dollar Volume'] = (tech_data.loc[:, 'Dollar Volume'].unstack('Symbol').rolling(5*12, min_periods=12).mean().stack())

tech_data['DV Rank'] = (tech_data.groupby('Date')['Dollar Volume'].rank(ascending=False))

# Filter top 150 most liquid stocks for each month and put in new dataframe
#tech_data = tech_data[tech_data['DV Rank']<150].drop(['Dollar Volume', 'DV Rank'], axis=1)
top_150 = tech_data[tech_data['DV Rank'] < 150]

top_150


In [ ]:
# Calculate monthly returns for different time horizons on the top 150 most liquid stocks
def calculate_returns(df):
    outlier_cutoff = 0.005
    lags = [1, 2, 3, 6, 9, 12]
    for lag in lags:
        df[f'{lag}m Return'] = (df['Adj Close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df

returns = top_150.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
returns

In [ ]:
# Use Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=start_date, end=end_date)[0].drop('RF', axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data.index.name = 'Date'
factor_data = factor_data.resample('M').last().div(100)
factor_data = factor_data.join(returns['1m Return']).sort_index()


# Filter out stocks with less than 10 months of data.
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >= 10]
factor_data = factor_data[factor_data.index.get_level_values('Symbol').isin(valid_stocks.index)]

factor_data


In [ ]:
# Calculate Rolling Factor Betas

betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['1m Return'], 
                                     exog=sm.add_constant(x.drop('1m Return', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

#betas 

# Join betas to returns dataframe
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
data = (returns.join(betas.groupby('Symbol').shift()))
data.loc[:, factors] = data.groupby('Symbol', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))
#data = data.drop(['Adj Close', 'Dollar Volume', 'DV Rank', axis=1)
data = data.dropna()

data.info()

In [ ]:
# K-Means Clustering for each month
from sklearn.cluster import KMeans
# N Clusters = 4
# Define initial centroids

target_rsi_values = [30, 45, 55, 70]
initial_centroids = np.zeros((len(target_rsi_values), 22))
initial_centroids[:, 6] = target_rsi_values


def get_clusters(df):
    df['Cluster'] = KMeans(n_clusters=4,
                           #random_state=0,
                           init=initial_centroids).fit(df).labels_
    return df

data = data.dropna().groupby('Date', group_keys=False).apply(get_clusters)

data


In [ ]:
# Plot Clusters
# def plot_clusters(df):
#     cluster0 = df[df['Cluster']==0]
#     cluster1 = df[df['Cluster']==1]
#     cluster2 = df[df['Cluster']==2]
#     cluster3 = df[df['Cluster']==3]

#     plt.scatter(cluster0['ATR'], cluster0['RSI'], color='red', label='Cluster 0')
#     plt.scatter(cluster1['ATR'], cluster1['RSI'], color='green', label='Cluster 1')
#     plt.scatter(cluster2['ATR'], cluster2['RSI'], color='blue', label='Cluster 2')
#     plt.scatter(cluster3['ATR'], cluster3['RSI'], color='yellow', label='Cluster 3')
    
#     plt.legend()
#     plt.show()
#     return 


def plot_clusters(df):
    fig, ax = plt.subplots(figsize=(10, 6))
    for i, cluster in df.groupby('Cluster'):
        cluster.plot.scatter(x='ATR', y='RSI', ax=ax, label=f'Cluster {i}', color=f'C{i}')
    return

# plt.style.use('ggplot')

for i in data.index.get_level_values('Date').unique().tolist():
    cluster = data.xs(i, level='Date')
    plt.title(f'Date {i.strftime("%Y-%m")}')
    plt.legend()
    plt.show()
    plot_clusters(cluster)
    

    


In [ ]:
# For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization
filtered_df = data[data['Cluster']==3].copy()

filtered_df = filtered_df.reset_index(level=1)

filtered_df.index = filtered_df.index+pd.DateOffset(1)

filtered_df = filtered_df.reset_index().set_index(['date', 'ticker'])

dates = filtered_df.index.get_level_values('date').unique().tolist()

fixed_dates = {}

for d in dates:
    
    fixed_dates[d.strftime('%Y-%m-%d')] = filtered_df.xs(d, level=0).index.tolist()
    
fixed_dates

In [ ]:
# Define portfolio optimization function

